In [1]:
from simplex import *

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")

## Постановка задачі

Розв'язати максимізаційну проблему:
\begin{equation}
\begin{cases}
z = 50x_1 + 80x_2 \rightarrow max;\\
x_1 + 2x_2 \leq 32;\\
3x_1 + 4x_2 \leq 84;\\
x_1, x_2 \geq 0
\end{cases}
\end{equation}

In [3]:
func = "50*x_1 + 80*x_2"
constr_less_1 = "1*x_1 + 2*x_2 <= 32"
constr_less_2 = "3*x_1 + 4*x_2 <= 84"

In [4]:
init_pr = convert_string_problem_into_arrays(func,
                                            [],
                                            [constr_less_1, constr_less_2],
                                            [],
                                            "")

In [5]:
init_pr

{'func': [50, 80],
 'equal': [],
 'less': [{'coefs': [1, 2], 'ind': [1, 2], 'val': 32},
  {'coefs': [3, 4], 'ind': [1, 2], 'val': 84}],
 'greater': [],
 'free_vars': []}

## Приведення до канонічної форми

Будь-яку задачу лінійного програмування можна звести до канонічної форми, яка має вигляд:
\begin{equation}
f(x) = <c, x> \rightarrow min;\\
Ax = b;\\
b \geq 0; x \geq 0,
\end{equation}
де \begin{equation}
A =\left( \begin{matrix}
a_{11} & \dots & a_{1n}\\
& \dots &\\
a_{m1} & \dots & a_{mn}
\end{matrix}\right),
\end{equation}
$n$ - кількість змінних, $m$ - кількість умов

Алгоритм переходу від довільної задачі ЛП до канонічної форми:

1. Нерівності з від'ємними $b_i$ множимо на $(-1)$;

2. Якщо нерівність вигляду "$\leq$", то до лівої частини додаємо $s_i \geq 0$ - додаткову змінну, і отримуємо рівність;

3. Якщо нерівність вигляду "$\geq$", то від лівої частини віднімаємо $s_j \geq 0$ - додаткову змінну, і отримуємо рівність;

4. Робимо заміну змінних:

* $x_i\leq 0 \Rightarrow x_i' = -x_i \geq 0$
* $x_i$ - будь-який, тоді $x_i = x_i' - x_i'', x_i', x_i'' \geq 0$

5. У задачі максимізації множимо коефіцієнти функції на $(-1)$

\begin{equation}
\begin{cases}
z = 50x_1 + 80x_2 \rightarrow max;\\
x_1 + 2x_2 \leq 32;\\
3x_1 + 4x_2 \leq 84;\\
x_1, x_2 \geq 0
\end{cases}
\end{equation}

In [6]:
A, b, c, add = canonic_form_of_problem(init_pr)

In [7]:
A

array([[1., 2., 1., 0.],
       [3., 4., 0., 1.]])

In [8]:
b

array([32., 84.])

In [9]:
c

array([-50., -80.,   0.,   0.])

## Перший крок симплекс методу

На першому кроці симплекс методу формується таблиця вигляду:

In [10]:
first_table, bases = simplex_init(A, b, c, add)

In [11]:
pd.DataFrame(first_table, columns = bases, index = ["z", "s_1", "s_2"])

,z,x_1,x_2,s_1,s_2,sol
z,1.0,-50.0,-80.0,0.0,0.0,0.0
s_1,0.0,1.0,2.0,1.0,0.0,32.0
s_2,0.0,3.0,4.0,0.0,1.0,84.0


## Розв'язок

In [12]:
table_final, res = simplex_method(first_table, bases)
pd.DataFrame(table_final, columns = bases, index = ["z", "x_2", "x_1"])

,z,x_1,x_2,s_1,s_2,sol
z,1.0,0.0,0.0,20.0,10.0,1480.0
x_2,0.0,0.0,1.0,1.5,-0.5,6.0
x_1,0.0,1.0,0.0,-2.0,1.0,20.0


In [13]:
res

{'z': 1480.0, 'x_1': 20.0, 'x_2': 6.0}

## Приклад, коли точка максимуму "не входить" в область

In [14]:
init_pr_1 = convert_string_problem_into_arrays("50*x_1+80*x_2",
                                            [],
                                            ["1*x_1+2*x_2 <= 32"],
                                            ["1*x_1+3*x_2 >= 50"],
                                            "")

In [15]:
A_1, b_1, c_1, add_1 = canonic_form_of_problem(init_pr_1)

In [16]:
first_table_1, bases_1 = simplex_init(A_1, b_1, c_1, add_1)

In [17]:
first_table_1

array([[  1., -50., -80.,   0.,   0.,   0.],
       [  0.,   1.,   2.,   1.,   0.,  32.],
       [  0.,   1.,   3.,   0.,  -1.,  50.]])

In [18]:
table_final_1, res_1 = simplex_method(first_table_1, bases_1)

In [19]:
pd.DataFrame(table_final_1, columns = bases)

,z,x_1,x_2,s_1,s_2,sol
0,1.0,0.0,20.0,50.0,0.0,1600.0
1,0.0,1.0,2.0,1.0,0.0,32.0
2,0.0,0.0,1.0,-1.0,-1.0,18.0


In [20]:
res_1

{'z': 1600.0, 'x_1': 32.0, 'x_2': 0}

### Приклад для 3 змінних

\begin{equation}
\begin{cases}
z = 200x_1 + 250x_2+100x_3 \rightarrow max;\\
x_1 + x_2+x_3 \leq 300;\\
30x_1 + 40x_2+50x_3 \leq 6400;\\
x_1+ x_2 + 2x_3 \leq 200;\\
x_1, x_2, x_3 \geq 0
\end{cases}
\end{equation}

In [21]:
init_pr_3d = convert_string_problem_into_arrays("200*x_1 + 250*x_2+100*x_3",
                                            [],
                                            ["1*x_1 + 1*x_2 + 1*x_3 <= 300",
                                            "30*x_1 + 40*x_2 + 50*x_3 <= 6400",
                                            "1*x_1 + 1*x_2 + 2*x_3 <= 200"],
                                            [],
                                            "")

In [22]:
A_3d, b_3d, c_3d, add_3d = canonic_form_of_problem(init_pr_3d)

In [23]:
first_table_3d, bases_3d = simplex_init(A_3d, b_3d, c_3d, add_3d)

In [24]:
print(first_table_3d)

[[ 1.0e+00 -2.0e+02 -2.5e+02 -1.0e+02  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  1.0e+00  1.0e+00  1.0e+00  1.0e+00  0.0e+00  0.0e+00  3.0e+02]
 [ 0.0e+00  3.0e+01  4.0e+01  5.0e+01  0.0e+00  1.0e+00  0.0e+00  6.4e+03]
 [ 0.0e+00  1.0e+00  1.0e+00  2.0e+00  0.0e+00  0.0e+00  1.0e+00  2.0e+02]]


In [25]:
table_final_3d, res_3d = simplex_method(first_table_3d, bases_3d)

In [26]:
pd.DataFrame(table_final_3d, columns = bases_3d)

,z,x_1,x_2,x_3,s_1,s_2,s_3,sol
0,1.0,0.0,0.0,250.0,0.0,5.0,50.0,42000.0
1,0.0,0.0,0.0,-1.0,1.0,0.0,-1.0,100.0
2,0.0,0.0,1.0,-1.0,0.0,0.1,-3.0,40.0
3,0.0,1.0,0.0,3.0,0.0,-0.1,4.0,160.0


In [27]:
res_3d

{'z': 42000.0, 'x_1': 160.0, 'x_2': 40.0, 'x_3': 0}

## Без обмеження на змінну

In [28]:
init_pr_3d_1 = convert_string_problem_into_arrays("200*x_1+250*x_2+100*x_3",
                                            [],
                                            ["1*x_1 + 1*x_2 + 1*x_3 <= 300",
                                            "30*x_1 + 40*x_2 + 50*x_3<=6400",
                                            "1*x_1 + 1*x_2 + 2*x_3 <= 200"],
                                            [],
                                            "x_3")

In [29]:
A_3d_1, b_3d_1, c_3d_1, add_3d_1 = canonic_form_of_problem(init_pr_3d_1)

In [30]:
first_table_3d_1, bases_3d_1 = simplex_init(A_3d_1, b_3d_1, c_3d_1, add_3d_1)

In [31]:
pd.DataFrame(first_table_3d_1, columns = bases_3d_1)

,z,x_1,x_2,x_3,x_4,s_1,s_2,s_3,sol
0,1.0,-200.0,-250.0,-100.0,100.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,-1.0,1.0,0.0,0.0,300.0
2,0.0,30.0,40.0,50.0,-50.0,0.0,1.0,0.0,6400.0
3,0.0,1.0,1.0,2.0,-2.0,0.0,0.0,1.0,200.0


In [32]:
table_final_3d_1, res_3d_1 = simplex_method(first_table_3d_1, bases_3d_1)

ArithmeticError: The problem has no optimal solution!